In [ ]:
!pip install validators

In [ ]:
import nltk
from nltk.corpus import twitter_samples,stopwords 
from nltk.tokenize import word_tokenize 
import string
import validators
import numpy as np
from nltk.stem import WordNetLemmatizer 
import re
nltk.download('wordnet') 
nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
ps = WordNetLemmatizer()

In [ ]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
tweets = all_positive_tweets + all_negative_tweets
labels = np.append(np.ones((len(all_positive_tweets),1)), np.zeros((len(all_negative_tweets),1)), axis = 0)
print(labels.shape)

In [55]:
def Process_Tweet(tweet):
  tweet = tweet.replace("n't"," not").replace("'m"," am").replace("'s"," is").replace("'ve"," have").replace("’","'").replace('`','')
  tweet = tweet.split()
  alphabets = "qwertyuioplkjhgfdsazxcvbnm"
  stop_words = set(stopwords.words('english'))
  tweet = [w.lower() for w in tweet if w not in stop_words and w[0] not in string.punctuation and not validators.url(w) and w[0] in alphabets]
  tweet = word_tokenize(" ".join(tweet))
  tweet = ["<s>"]+[re.sub(r'(.)\1{2,}', r'\1', w) for w in tweet if w not in string.punctuation] +["<\s>"]
  tweet = nltk.pos_tag(tweet)
  tweet = [(ps.lemmatize(w[0]),w[1]) for w in tweet]
  return tweet

In [ ]:
processsed_i = []
X = []
Y = []
postive_list = []
negative_list = []
vocab = set()
for i in range(len(tweets)):
  processsed_i = Process_Tweet(tweets[i])
  print("\rTweets",str(i+1),":",processsed_i,end="")
  if processsed_i != []:
    vocab.update(processsed_i)
    if labels[i]==1:
      postive_list = postive_list + processsed_i
      Y.append(1)
    else:
      Y.append(0)
      negative_list = negative_list + processsed_i
    X.append(processsed_i)
print("\nNumber of tweets:", len(X),len(Y))
vocab = list(vocab)
print("\nVocab Size:",len(vocab))

In [ ]:
positive_count = dict()
negative_count = dict()
for j in range(len(vocab)):
  i = vocab[j]
  print("\r",j,"/",len(vocab),end="")
  if postive_list.count(i)>0:
    positive_count[i] = postive_list.count(i)
  if negative_list.count(i)>0:
    negative_count[i] = negative_list.count(i)


In [ ]:
Processed_X = []
Processed_Y = []
length = 15
list_X = []
actual_X = []
for x_val in range(len(X)):
  i = X[x_val]
  k = 0
  pos_list = [0]*length
  neg_list = [0]*length
  if len(i)<length:
    for x in range(1,len(i)-1):
        j = i[x]
        if j[1] == 'NNP':
          continue
        if j in positive_count.keys():
            pos_list[k] += positive_count[j]
        if j in negative_count.keys():
            neg_list[k] += negative_count[j]
        k+=1
    list_X.append(X[x_val])
    actual_X.append(tweets[x_val])
    Processed_X.append([pos_list,neg_list])
    Processed_Y.append(Y[x_val])

In [ ]:
X_array = np.array(Processed_X)#.reshape((len(Processed_X),length,2))
X_array = np.transpose(X_array,axes=(0,2,1))
X_array.shape
val = 19
print("Actual Tweet:",actual_X[val])
print("After preprocessing:",list_X[val])
print("Representation:\n",X_array[val])
Y_array = np.array(Processed_Y).reshape((len(Processed_Y),1))
print("Label:",Y_array[val])

In [62]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Reshape
from keras.optimizers import Adam

In [63]:
def create_model(inp_shape,batch_size):
    model = Sequential()
    model.add(LSTM(512, input_shape = inp_shape, activation = 'relu'))
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2048,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2048,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation='sigmoid'))
    return model

In [ ]:
BATCH_SIZE = 32
model = create_model(X_array[0].shape,BATCH_SIZE)
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_array,Y_array,train_size=0.8, random_state=7)
x_train.shape

In [ ]:
opt = Adam(learning_rate=0.00005)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x=x_train,y=y_train,epochs=35,validation_split=0.1,batch_size=BATCH_SIZE,validation_steps=20,shuffle=True)

In [ ]:
history = model.fit(x=x_train,y=y_train,epochs=35,validation_split=0.1,batch_size=BATCH_SIZE,validation_steps=20,shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'Val'], loc='upper left')
plt.show()

In [ ]:
Y_pred = model.predict(x_test)
print(Y_pred)
print(Y_pred.shape)
Y_pred = np.round(Y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, Y_pred, labels=[0,1])

In [ ]:
acc = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])
print(acc)